<a href="https://colab.research.google.com/github/felippecaso/intensivo-dados-mentorama/blob/main/pratica_carreiras_ferramentas_dados_mentorama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>